In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly import data

In [2]:
tips = data.tips()
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
# how to make a custom sort using pd.Categorical
pd.Categorical(["a", "b", "a", "a", "c"], categories=["c", "b", "a"], ordered=True)

['a', 'b', 'a', 'a', 'c']
Categories (3, object): ['c' < 'b' < 'a']

In [20]:
day_order = ['Thur', 'Fri', 'Sat', 'Sun']
agg_tips = tips.groupby('day').agg(
    total_bill=('total_bill', 'mean'),
    tip=('tip', 'mean'),
)
agg_tips.index = pd.Categorical(agg_tips.index, categories=day_order, ordered=True)
agg_tips = agg_tips.sort_index()
agg_tips

,total_bill,tip
Thur,17.682742,2.771452
Fri,17.151579,2.734737
Sat,20.441379,2.993103
Sun,21.410000,3.255132


In [25]:
fig = go.Figure()

for col in agg_tips.columns:
    fig.add_trace(
        go.Scatter(x=agg_tips.index.astype(str), y=agg_tips[col], mode='lines+markers', name=col)
    )

fig.add_hline(y=10, line_dash="dash", line_color="black", annotation_text="Ref Line")

fig.update_layout(
    title='Daily Average Tips and Bills',
    xaxis_title='Day of Week',
    yaxis_title='Amount ($)',
    width=1200,
    height=400,
)

fig.show()


In [30]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

df = px.data.iris()
species = df['species'].unique()

fig = make_subplots(rows=1, cols=3, subplot_titles=species, 
                    shared_xaxes=True, shared_yaxes=True)

for i, sp in enumerate(species, 1):
    data = df[df['species'] == sp]
    fig.add_trace(
        go.Scatter(x=data['sepal_width'], y=data['sepal_length'], mode='markers'),
        row=1, col=i
    )

fig.update_xaxes(rangemode="tozero", title_text="Sepal Width")
fig.update_yaxes(rangemode="tozero", title_text="Sepal Length", row=1, col=1)
fig.show()


In [27]:
import plotly.express as px
df = px.data.iris()

fig = px.scatter(df, x="sepal_width", y="sepal_length", facet_col="species")
fig.update_xaxes(rangemode="tozero")
fig.update_yaxes(rangemode="tozero")

fig.show()

In [31]:
# Prepare data with min and max
df = tips.groupby('day')['total_bill'].agg(['min', 'max']).reset_index()

fig = go.Figure()

# Add lines connecting min to max
for i, row in df.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['min'], row['max']], 
        y=[row['day'], row['day']],
        mode='lines',
        line=dict(color='gray', width=2),
        showlegend=False
    ))

# Add min points
fig.add_trace(go.Scatter(
    x=df['min'], y=df['day'], 
    mode='markers', 
    marker=dict(size=10, color='blue'),
    name='Min'
))

# Add max points
fig.add_trace(go.Scatter(
    x=df['max'], y=df['day'], 
    mode='markers', 
    marker=dict(size=10, color='red'),
    name='Max'
))

fig.show()


In [32]:
# Example: Hyperparameter tuning results
df = pd.DataFrame({
    'learning_rate': [0.01, 0.001, 0.1, 0.05, 0.01],
    'max_depth': [3, 5, 3, 7, 5],
    'n_estimators': [100, 200, 150, 100, 250],
    'accuracy': [0.85, 0.92, 0.78, 0.88, 0.94]
})

fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = df['accuracy'],
                   colorscale = 'Viridis',
                   showscale = True,
                   cmin = df['accuracy'].min(),
                   cmax = df['accuracy'].max()),
        dimensions = [
            dict(label = 'Learning Rate', values = df['learning_rate']),
            dict(label = 'Max Depth', values = df['max_depth']),
            dict(label = 'N Estimators', values = df['n_estimators']),
            dict(label = 'Accuracy', values = df['accuracy'])
        ]
    )
)

fig.show()


In [33]:
fig = go.Figure(go.Funnel(
    y = ["Website Visit", "Add to Cart", "Checkout", "Purchase"],
    x = [1000, 500, 300, 150],
    textinfo = "value+percent initial"
))

fig.update_layout(title="Sales Funnel")
fig.show()


In [ ]:
# think it as node, edge, weight
# but we need to have node id to control
fig = go.Figure(data=[go.Sankey(
    node = dict(
      label = ["Website", "Mobile App", "Purchased", "Abandoned", "Returned"]
    ),
    link = dict(
      source = [0, 0, 1, 1, 2],
      target = [2, 3, 2, 3, 4],
      value = [100, 50, 80, 20, 15]
  ))])

fig.update_layout(title="Customer Journey")
fig.show()


In [39]:
# Prepare data
df = pd.DataFrame({
    'category': ['A', 'B', 'C', 'D', 'E'],
    'value': [40, 30, 15, 10, 5]
})
df = df.sort_values('value', ascending=False)
df['cumulative'] = df['value'].cumsum() / df['value'].sum() * 100

# Create figure with secondary y-axis
fig = go.Figure()

# Bar chart
fig.add_trace(go.Bar(
    x=df['category'], 
    y=df['value'], 
    name='Value',
    yaxis='y'
))

# Cumulative line
fig.add_trace(go.Scatter(
    x=df['category'], 
    y=df['cumulative'], 
    name='Cumulative %',
    mode='lines+markers',
    yaxis='y2'
))

# Layout with dual axes
fig.update_layout(
    yaxis=dict(title='Value'),
    yaxis2=dict(title='Cumulative %', overlaying='y', side='right', range=[0, 110])
)

fig.show()
